In [1]:
import caiman as cm
from caiman.source_extraction.cnmf.cnmf import load_CNMF
from caiman.utils.visualization import get_contours as caiman_get_contours
import numpy as np

2022-05-30 18:18:52.986674: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/arjun/miniconda3/envs/core_test/lib/python3.9/site-packages/cv2/../../lib64:/usr/lib/cuda/include:/usr/lib/cuda/lib64:
2022-05-30 18:18:52.986718: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from mesmerize_core import *
from mesmerize_napari import *
from mesmerize_core import CNMFExtensions

In [3]:
df = load_batch('/home/arjun/CaImAn/example_movies/test_5_7.pickle')

In [4]:
df

,algo,name,input_movie_path,params,outputs,uuid
0,mcorr,asdf,CaImAn/example_movies/demoMovie.tif,"{'mcorr_kwargs': {'max_shifts': [6, 6], 'niter...",{'mcorr-output-path': CaImAn/example_movies/1d...,1da21ea4-f45f-47d1-b8ab-047cb51468c2
1,cnmf,asdf,CaImAn/example_movies/demoMovie.tif,"{'refit': True, 'cnmf_kwargs': {'p': 2, 'nb': ...",{'cnmf-hdf5-path': CaImAn/example_movies/92a8f...,92a8f762-b9e6-4ea6-a1b5-d46138c04112
2,cnmf,asdf_2,CaImAn/example_movies/92a8f762-b9e6-4ea6-a1b5-...,"{'refit': True, 'cnmf_kwargs': {'p': 2, 'nb': ...",{'cnmf-hdf5-path': CaImAn/example_movies/2b27b...,2b27b4ff-e305-4865-89fd-d68a49bc2467
3,mcorr,asdf_pw,CaImAn/example_movies/demoMovie.tif,"{'mcorr_kwargs': {'max_shifts': [6, 6], 'niter...",{'mean-projection-path': CaImAn/example_movies...,1e1df34c-0ce7-48a7-8154-4334e620ff71
4,cnmf,asdf,CaImAn/example_movies/ff6a043b-6bae-4628-b56c-...,"{'refit': True, 'cnmf_kwargs': {'p': 2, 'nb': ...",{'mean-projection-path': CaImAn/example_movies...,396c7269-9799-44d1-8a55-1a9d9da86d44
5,cnmf,fr_test,CaImAn/example_movies/1e1df34c-0ce7-48a7-8154-...,"{'refit': True, 'cnmf_kwargs': {'p': 2, 'nb': ...",{'mean-projection-path': CaImAn/example_movies...,f7f6f4f6-6fef-4ea1-9afc-b33b9de33b61
6,mcorr,test_qprocess,CaImAn/example_movies/demoMovie.tif,"{'mcorr_kwargs': {'max_shifts': [6, 6], 'niter...",None,c2257852-d0ec-450b-9ec1-ac5a1884e7fd
7,cnmf,asdf,CaImAn/example_movies/1e1df34c-0ce7-48a7-8154-...,"{'refit': True, 'cnmf_kwargs': {'p': 2, 'nb': ...",None,11407c48-e04b-4ac6-bdf0-220288876800


In [5]:
set_parent_data_path('/home/arjun/')

PosixPath('/home/arjun')

In [6]:
cnmf_obj = df.iloc[1].cnmf.get_output()

In [7]:
cnmf_obj.estimates.idx_components

array([0, 1, 2, 3, 4, 5, 7, 8, 9])

In [8]:
cnmf_obj.estimates.idx_components_bad

array([6])

In [9]:
some_ixs = np.array([1, 3, 6, 9])

In [10]:
df.iloc[1].cnmf._get_spatial_contours(
    cnmf_obj=cnmf_obj, 
    ixs_components=cnmf_obj.estimates.idx_components
)

TypeError: unhashable type: 'numpy.ndarray'